In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import numpy as np
from conversion import ECIS_SUMMER_2020, ALL_SITE_FILE, \
    COLUMN_MAPPING_ECIS_ECE, ECE_COLUMNS, RACE_COLS, EXIT_REASONS, FUNDING_TYPE, SPACE_TYPE_LOOKUP, get_age_group

In [4]:
summer_df = pd.read_csv(ECIS_SUMMER_2020,sep='\t', dtype={'ZipCode':'str','AssignedFacilityCode':'str'})
summer_df = summer_df.rename(columns=COLUMN_MAPPING_ECIS_ECE)

In [5]:
# Drop funding types that aren't in the new ECE system
FUNDING_TYPES_TO_DROP = ['PDG-Federal','Private Pay','Head Start/Early Head Start']
summer_df = summer_df[~summer_df['Funding type'].isin(FUNDING_TYPES_TO_DROP)]

In [ ]:
Aged out, 
Moved within program, 
Stopped attending, 
Chose to attend a 
different program,  
Moved within 
Connecticut, Moved to 
another state, 
Withdrew due to lack 
of payment, Child was 
asked to leave, 
Unknown 

In [6]:
summer_df['Race not disclosed'] = ~summer_df[RACE_COLS].replace({'Y':True,'N':False}).any(axis=1)
summer_df['Hispanic or Latinx ethnicity'].fillna('Not collected', inplace=True)
summer_df['Gender'].fillna('Not Specified', inplace=True)
summer_df['Receiving disability and/or special education services'].fillna('Not collected', inplace=True)
summer_df['Street address'] = summer_df['StreetAddress1'] + summer_df['StreetAddress2'].fillna('')
summer_df['Lives with foster family'] = summer_df['WhoChildResidesWith'] == 'Foster Parent'
summer_df['Enrollment exit reason'].fillna('',inplace=True)
summer_df['Enrollment exit reason'].replace(EXIT_REASONS, inplace=True)
summer_df['Funding type'].replace(FUNDING_TYPE, inplace=True)


# Calculate Space type and age group
summer_df['funding_and_space_type'] = (summer_df['Funding type'] + ' || ' + summer_df['SpaceType'])
summer_df['Space type'] = summer_df['funding_and_space_type'].replace(SPACE_TYPE_LOOKUP)
summer_df['Age group'] = summer_df['funding_and_space_type'].apply(get_age_group)

In [7]:
numeric_cols = ['Household size', 'Annual household income', 'Determination date']
for col in numeric_cols:
    summer_df[col].fillna('', inplace=True)

In [8]:
date_cols = ['First funding period', 'Last funding period','Enrollment start date','Enrollment end date']
for col in date_cols:
    summer_df[col] = pd.to_datetime(summer_df[col]).dt.strftime('%m/%d/%Y')

In [9]:
summer_df['Care model'] = 'Unknown'

In [10]:
all_sites_df = pd.read_csv(ALL_SITE_FILE)
summer_df = summer_df.merge(all_sites_df, how='left', left_on='AssignedFacilityCode', right_on='Facility Code')

In [11]:
# 12 sites don't exist in our current set of sites with the ECIS IDs, this fills in the best guess for organization
# A new site may need to be added
site_to_org_mapping = {'Lulac': 'City of New Haven',
                       'Saint Aedan Elementary School':'Catholic Academy of New Haven-St. Aedan Pre School',
                       'Hospital for Special Care':'Hospital for Special Care',
                       'Bright and Early Children Learning Center':"Bright and Early Children's Learning Centers I",
                       'Gallup Hill School':'Gallup Hill School',
                       'Early Childhood - CT Office of':'Early Childhood - CT Office of',
                       'Alice Peck Learning Center':'Hamden BOE',
                       'TVCCA Groton ECE':'Thames Valley Council for Comm. Action TVCCA)',
                       'Hockanum Pre-K':'East Hartford BOE',
                       'WCSU Child Care Center':'WCSU Child Care Center',
                       'Colchester Early Childhood':'Colchester BOE',
                       'Precious Memories lll':'Precious Memories Early Childhood Learning Center'                       
                      }
summer_df['Provider'] = summer_df['Parent Org'].fillna(summer_df['AssignedFacilityName']).replace(site_to_org_mapping)
# Use canonical site names where they exist, ie. all but the above 12 sites
summer_df['Site'] = summer_df['Site Name'].fillna(summer_df['AssignedFacilityName'])

In [20]:
missing_ece_columns = ['Dual language learner', 'Experiencing homelessness or housing insecurity', 'Birth certificate type']

final_df = summer_df[[x for x in ECE_COLUMNS if x not in missing_ece_columns]].sort_values(['Provider','Site'])
final_df.to_csv('data/student_data.csv', index=False)

## Data checks

In [38]:
pd.unique(summer_df[RACE_COLS].values.ravel())

array(['N', 'Y'], dtype=object)

In [ ]:
null_race
street_address
foster_family
funding_start
funding_end
site
provider
enrollment_start
enrollment_end
space_type